In [1]:
import numpy as np
import cv2

In [2]:
class Sketcher:
    def __init__(self, windowname, dests, colors_func):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False
        cv2.setMouseCallback(self.windowname, self.on_mouse)
        
    def show(self):
        cv2.imshow(self.windowname, self.dests)
        
    def on_mouse(self, event, x, y, flags, param):
        pt = (x, y)
        if event == cv2.EVENT_LBUTTONDOWN:
            self.prev_pt = pt
        elif event == cv2.EVENT_LBUTTONUP:
            self.prev_pt = None
        
        if self.prev_pt and flags & cv2.EVENT_FLAG_LBUTTON:
            self.dirty = True
            self.prev_pt = pt
            self.show()

In [ ]:
img = cv2.imread("blemish.png", cv2.IMREAD_COLOR)